# 1. annotate vcf file

the location of data on genomDK is
/home/joco/faststorage/soyadapt_data_analysis_with_start_data/resources/start_data


i will work with SNPeff on genomedk
for this i will make a seperate projhect folder as the otherfolders are backups of the rest of my project from my laptop+github 

snpeff/
   README.txt
   resources/
       README.txt
       ...
   steps/
       README.txt
       ...
   results/
       README.txt
       ...
   plots/
       README.txt
       ...
   workflow/
       README.txt
       notebooks
       envs
         environment.yml


conda activate

conda create -n snpeffenv 

Download latest version
wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip

Unzip file
unzip snpEff_latest_core.zip

i had to update to java 11 on the environment only



In [ ]:
#conda install -c conda-forge openjdk=11
java -version

#local !!! You have to be in the unzipped /snpEff file to run the following command! !!

#on genome dk go to 
cd /home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/
#found its location after downlading via conda with
# find / -name snpEff.jar 2>/dev/null

java -jar snpEff.jar databases | grep -i glycine_max

Glycine_max                                                 	Glycine_max                                                 	          	                              	[https://snpeff.blob.core.windows.net/databases/v5_1/snpEff_v5_1_Glycine_max.zip, https://snpeff.blob.core.windows.net/databases/v5_0/snpEff_v5_0_Glycine_max.zip]

 java -jar snpEff.jar download Glycine_max

collect the files

1. path to reference genome file (fasta) - the same as used for VCF construction

2. path to Reference genome gene annotation (gff) :

3. path to output directory

4. path to vcf input
5. the downloaded database data



in genomeDK:
1. ref=/home/joco/faststorage/snpeff_analysis/resources/Wm82.a2.v1.genome.fasta
2. ann=/home/joco/faststorage/snpeff_analysis/resources/Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz

3. out=/home/joco/faststorage/snpeff_analysis/resources/generated_data/
4. wgs_maf5=/home/joco/faststorage/snpeff_analysis/resources/start_data/wgs_maf5.vcf.gz
5. snpeff_data=/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/Glycine_max


i checked to see if the chromsomes have the same name in my vcf file as the .fasta and .gff3



zcat Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz | cut -f1 | sort | uniq

fasta file

zcat your_file.fasta.gz | grep "^>" | cut -d " " -f 1 | sed 's/>//'

so maybe not the same 



# run snpeff with database

In [ ]:
in:
/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/

export wgs_maf5=/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/wgs_maf5.vcf.gz

java -Xmx8g -jar snpEff.jar -v Glycine_max  $wgs_maf5 > wgs_maf5.ann.vcf.gz
# i will make one more with stats!
java -Xmx8g -jar snpEff.jar -v -stats ex1.html Glycine_max $wgs_maf5 > 2_wgs_maf5.ann.vcf.gzann.vcf

In [ ]:
#great i now have a file:
/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/wgs_maf5.ann.vcf.gz



i now want to compare the windows with differing values of signatures of selection in the list of genes against the rest of the genome

i need a Gene List: Ensure you have a list of genes you want to compare against the rest of the genome. This list could be in a simple text file, with one gene symbol per line.

Gene Coordinates: Obtain the genomic coordinates (start and end positions) of each gene in your gene list. 


    Calculate Average Signature Value: For each gene in your list, calculate the average value of signatures of selection across the windows that overlap with the gene's coordinates. You can write a script or use bioinformatics tools to do this.

    Calculate Average Value for Rest of Genome: Calculate the average value of signatures of selection for all the windows in the rest of the genome (i.e., excluding the windows that overlap with the genes in your list).

    Statistical Comparison: Perform a statistical comparison, such as a t-test or Wilcoxon rank-sum test, to determine if the average signature value in your gene list is significantly different from the average value in the rest of the genome.

    Multiple Testing Correction (Optional): If you are comparing against a large number of windows, consider applying multiple testing correction methods, such as Bonferroni or False Discovery Rate (FDR) correction, to account for the increased risk of false positives.

    Visualization: You can create visualizations to compare the average signature values between your gene list and the rest of the genome. Box plots, violin plots, or bar plots can be useful for this purpose.

Keep in mind that bioinformatics analysis can be complex, and the specific tools and scripts you'll use may depend on the format of your data and the programming language you are comfortable with. If you are not familiar with bioinformatics analyses, it might be helpful to seek assistance from a bioinformatician or collaborator with expertise in this area. Additionally, various bioinformatics software tools like R packages or Python libraries can aid in data manipulation, analysis, and visualization during this process.

bcftools view -h wgs_maf5.ann.vcf.gz   # View the VCF header
#checked out the annotation format
bcftools view wgs_maf5.ann.vcf.gz | less


i have a list of genes to try out


Glyma01g02520
Glyma01g02530
Glyma01g02880
Glyma01g05960
Glyma01g07070
Glyma01g08130
Glyma01g08150
Glyma01g11670
Glyma01g21710
Glyma01g22260
Glyma01g36730
Glyma01g36810
Glyma01g37190
Glyma01g37470
Glyma01g39490
Glyma01g39520
Glyma01g39950
Glyma01g41460
Glyma01g43360
Glyma02g00830
Glyma02g00980
Glyma02g01540
Glyma02g01760
Glyma02g04710
Glyma02g05100
Glyma02g07650
Glyma02g09600
Glyma02g09860
Glyma02g10880
Glyma02g11060
Glyma02g12080
Glyma02g12930
Glyma02g13400
Glyma02g13420
Glyma02g14880
Glyma02g15850
Glyma02g17110
Glyma02g18610
Glyma02g29380
Glyma02g33040
Glyma02g36210
Glyma02g37120
Glyma02g37880
Glyma02g38090
Glyma02g43540
Glyma02g43760
Glyma02g45730
Glyma02g46680
Glyma02g46970
Glyma03g00580
Glyma03g01030
Glyma03g02200
Glyma03g02210
Glyma03g22710
Glyma03g23260
Glyma03g27010
Glyma03g31670
Glyma03g33470
Glyma03g33490
Glyma03g35250
Glyma03g35610
Glyma03g36970
Glyma03g37840
Glyma03g38320
Glyma03g38390
Glyma03g38620
Glyma03g38670
Glyma03g39910
Glyma03g40840
Glyma03g42220
Glyma03g42260
Glyma04g00340
Glyma04g01400
Glyma04g02420
Glyma04g02980
Glyma04g04170
Glyma04g05280
Glyma04g06240
Glyma04g07540
Glyma04g07900
Glyma04g08680
Glyma04g09820
Glyma04g10820
Glyma04g11010
Glyma04g14840
Glyma04g31800
Glyma04g31810
Glyma04g32540
Glyma04g33110
Glyma04g33410
Glyma04g35810
Glyma04g36620
Glyma04g36630
Glyma04g37290
Glyma04g37900
Glyma04g38060
Glyma04g38860
Glyma04g40640
Glyma04g42410
Glyma04g42420
Glyma04g43620
Glyma04g43640
Glyma05g00880
Glyma05g00970
Glyma05g01510
Glyma05g03660
Glyma05g07350
Glyma05g07380
Glyma05g13890
Glyma05g18170
Glyma05g22250
Glyma05g22320
Glyma05g25200
Glyma05g26150
Glyma05g26460
Glyma05g28130
Glyma05g28140
Glyma05g29090
Glyma05g29590
Glyma05g31680
Glyma05g32680
Glyma05g34030
Glyma05g34280
Glyma05g34530
Glyma05g35280
Glyma05g37070
Glyma06g00400
Glyma06g01430
Glyma06g02470
Glyma06g02990
Glyma06g04350
Glyma06g06300
Glyma06g08790
Glyma06g09890
Glyma06g10020
Glyma06g10650
Glyma06g10830
Glyma06g12380
Glyma06g12390
Glyma06g14150
Glyma06g17010
Glyma06g17170
Glyma06g17780
Glyma06g18290
Glyma06g18300
Glyma06g20950
Glyma06g21120
Glyma06g21900
Glyma06g22650
Glyma06g26590
Glyma06g37080
Glyma06g37220
Glyma06g38600
Glyma06g47220
Glyma06g48270
Glyma07g01600
Glyma07g04180
Glyma07g05410
Glyma07g05530
Glyma07g06420
Glyma07g08820
Glyma07g08830
Glyma07g08890
Glyma07g08920
Glyma07g09800
Glyma07g09990
Glyma07g21160
Glyma07g25970
Glyma07g30040
Glyma07g31880
Glyma07g33600
Glyma07g35240
Glyma07g35690
Glyma07g37640
Glyma07g37770
Glyma07g39350
Glyma08g00330
Glyma08g02490
Glyma08g04440
Glyma08g05130
Glyma08g05390
Glyma08g05650
Glyma08g06980
Glyma08g07260
Glyma08g08220
Glyma08g09090
Glyma08g09380
Glyma08g11110
Glyma08g11120
Glyma08g12230
Glyma08g12730
Glyma08g14930
Glyma08g15700
Glyma08g17630
Glyma08g19590
Glyma08g21110
Glyma08g22400
Glyma08g24340
Glyma08g24550
Glyma08g24590
Glyma08g27670
Glyma08g27680
Glyma08g28370
Glyma08g28470
Glyma08g36380
Glyma08g42300
Glyma08g43760
Glyma08g44140
Glyma08g44640
Glyma08g44650
Glyma08g47810
Glyma08g47820
Glyma08g47900
Glyma09g01000
Glyma09g01650
Glyma09g03990
Glyma09g04970
Glyma09g05080
Glyma09g05090
Glyma09g06220
Glyma09g06750
Glyma09g07120
Glyma09g07240
Glyma09g07770
Glyma09g11600
Glyma09g17220
Glyma09g18790
Glyma09g20280
Glyma09g26550
Glyma09g27450
Glyma09g28670
Glyma09g29030
Glyma09g31770
Glyma09g32010
Glyma09g37170
Glyma09g38290
Glyma09g40230
Glyma09g42060
Glyma09g42120
Glyma09g42140
Glyma10g01580
Glyma10g01830
Glyma10g02690
Glyma10g03910
Glyma10g05520
Glyma10g05610
Glyma10g08340
Glyma10g08370
Glyma10g08710
Glyma10g22390
Glyma10g23370
Glyma10g23420
Glyma10g28170
Glyma10g28290
Glyma10g29690
Glyma10g31190
Glyma10g31450
Glyma10g32390
Glyma10g33550
Glyma10g34760
Glyma10g35350
Glyma10g36600
Glyma10g36820
Glyma10g38580
Glyma10g43190
Glyma11g02110
Glyma11g03950
Glyma11g03960
Glyma11g05340
Glyma11g05720
Glyma11g05760
Glyma11g07150
Glyma11g07820
Glyma11g08090
Glyma11g09700
Glyma11g10670
Glyma11g13210
Glyma11g13220
Glyma11g13490
Glyma11g14490
Glyma11g15560
Glyma11g15580
Glyma11g15650
Glyma11g16110
Glyma11g18190
Glyma11g20200
Glyma11g29610
Glyma11g29860
Glyma11g35270
Glyma11g36890
Glyma12g02980
Glyma12g03700
Glyma12g04670
Glyma12g05250
Glyma12g05490
Glyma12g06410
Glyma12g07800
Glyma12g07860
Glyma12g10320
Glyma12g13560
Glyma12g17720
Glyma12g25240
Glyma12g30940
Glyma12g30980
Glyma12g34240
Glyma12g34510
Glyma12g35320
Glyma13g00510
Glyma13g00860
Glyma13g00920
Glyma13g01290
Glyma13g01810
Glyma13g02310
Glyma13g03880
Glyma13g05480
Glyma13g06730
Glyma13g07030
Glyma13g09660
Glyma13g10690
Glyma13g16700
Glyma13g16850
Glyma13g19870
Glyma13g22030
Glyma13g22060
Glyma13g24530
Glyma13g24590
Glyma13g26820
Glyma13g27770
Glyma13g27780
Glyma13g27790
Glyma13g29510
Glyma13g30330
Glyma13g31050
Glyma13g31090
Glyma13g33040
Glyma13g35190
Glyma13g35980
Glyma13g36310
Glyma13g39340
Glyma13g39360
Glyma13g40470
Glyma13g41140
Glyma13g42060
Glyma13g42660
Glyma14g03100
Glyma14g05150
Glyma14g05430
Glyma14g06480
Glyma14g07960
Glyma14g10530
Glyma14g17260
Glyma14g21260
Glyma14g24590
Glyma14g33810
Glyma14g35020
Glyma14g35410
Glyma14g36100
Glyma14g36220
Glyma14g37280
Glyma15g02770
Glyma15g03330
Glyma15g04260
Glyma15g04930
Glyma15g05440
Glyma15g06300
Glyma15g06310
Glyma15g06470
Glyma15g06490
Glyma15g08270
Glyma15g08320
Glyma15g08860
Glyma15g09500
Glyma15g11850
Glyma15g12570
Glyma15g14980
Glyma15g15730
Glyma15g15890
Glyma15g17480
Glyma15g18010
Glyma15g18050
Glyma15g18450
Glyma15g19390
Glyma15g23400
Glyma15g23610
Glyma15g29870
Glyma15g35080
Glyma15g37830
Glyma16g00890
Glyma16g01980
Glyma16g02050
Glyma16g03030
Glyma16g04830
Glyma16g04840
Glyma16g05110
Glyma16g08860
Glyma16g13070
Glyma16g17450
Glyma16g26660
Glyma16g26690
Glyma16g32080
Glyma16g32540
Glyma17g01370
Glyma17g02860
Glyma17g03010
Glyma17g03960
Glyma17g05860
Glyma17g05990
Glyma17g06640
Glyma17g06950
Glyma17g07000
Glyma17g07420
Glyma17g08890
Glyma17g10370
Glyma17g10920
Glyma17g11040
Glyma17g14190
Glyma17g17520
Glyma17g17790
Glyma17g18640
Glyma17g29720
Glyma17g34980
Glyma17g37050
Glyma18g00800
Glyma18g03130
Glyma18g06460
Glyma18g07770
Glyma18g08620
Glyma18g09060
Glyma18g11180
Glyma18g11400
Glyma18g12590
Glyma18g33910
Glyma18g45780
Glyma18g49520
Glyma18g50900
Glyma18g50910
Glyma18g51320
Glyma18g53610
Glyma18g53670
Glyma18g53680
Glyma18g53690
Glyma19g02710
Glyma19g04320
Glyma19g05170
Glyma19g20090
Glyma19g27950
Glyma19g28390
Glyma19g28400
Glyma19g29610
Glyma19g30230
Glyma19g34470
Glyma19g36200
Glyma19g36220
Glyma19g37890
Glyma19g37910
Glyma19g38230
Glyma19g39620
Glyma19g40430
Glyma19g40980
Glyma19g41210
Glyma19g41260
Glyma19g42460
Glyma19g43540
Glyma19g44970
Glyma19g45030
Glyma20g00350
Glyma20g00400
Glyma20g01130
Glyma20g03120
Glyma20g04600
Glyma20g10600
Glyma20g16880
Glyma20g19600
Glyma20g22160
Glyma20g22280
Glyma20g23700
Glyma20g24220
Glyma20g29250
Glyma20g30980
Glyma20g32160
Glyma20g32730
Glyma20g34050
Glyma20g35220
Glyma20g36110
Glyma20g36330
Glyma20g37620
Glyma20g39140

In [ ]:
# something java -jar snpEff.jar  floweringgenes > flowering_gene_positions.txt



In [ ]:
bcftools query -f '%CHROM\t%POS\t%INFO/ANN\n' wgs_maf5.ann.vcf.gz > output.txt
#im looking at ANN
# 

##SnpEffVersion="5.1 (build 2022-01-21 06:23), by Pablo Cingolani"
##SnpEffCmd="SnpEff  Glycine_max /home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/wgs_maf5.vcf.gz "
##INFO=<ID=ANN,Number=.,Type=String,Description="Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO'">
##INFO=<ID=LOF,Number=.,Type=String,Description="Predicted loss of function effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">
##INFO=<ID=NMD,Number=.,Type=String,Description="Predicted nonsense mediated decay effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">

1	27	G|intergenic_region|MODIFIER|CHR_START-GLYMA_01G000100|CHR_START-GLYMA_01G000100|intergenic_region|CHR_START-GLYMA_01G000100|||n.27A>G||||||
so $4


In [ ]:
#i moved loval to 
/Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data

In [ ]:
sed 's/|/\t/g' output.txt | awk -F '\t' '{print $1, $2, $4, $5, $6, $7}' | awk -F '\t' '{print $1, $2, tolower($4), tolower($5),tolower($6), tolower($7)}' > extracted_genes.txt

notes
scans for quality
stats?
https://github.com/alexherbig/MultiVCFAnalyzer#usage